# Practica 2
## Diego Rodríguez y Alejandro Jiménez
## Grupo 1463

In [2]:
from abc import ABCMeta,abstractmethod
import pandas as pd
import numpy as np
from Datos import *
from EstrategiaParticionado import *
from Clasificador import *
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB


# Implementación Propia
## Tic-tac-toe

In [6]:
datos = Datos('./ConjuntoDatos/tic-tac-toe.data')
# Validacion simple con 100 iteraciones
vs = ValidacionSimple(0.30, 100)
# Validación simple con 60 folds
vc = ValidacionCruzada(60)
# Creación del clasificador
nb = ClasificadorNaiveBayes()
# Cálculo de errores
mio_tic_simple = nb.validacion(vs, {"nombre": "naiveBayes", "laplace": False}, datos)
print("Error con validación simple sin laplace: " + str(mio_tic_simple))
mio_tic_simple_laplace = nb.validacion(vs, {"nombre": "naiveBayes", "laplace": True}, datos)
print("Error con validación simple con laplace: " + str(mio_tic_simple_laplace))
mio_tic_cruzada = nb.validacion(vc, {"nombre": "naiveBayes", "laplace": False}, datos)
print("Error con validación cruzada sin laplace: " + str(mio_tic_cruzada))
mio_tic_cruzada_laplace = nb.validacion(vc, {"nombre": "naiveBayes", "laplace": True}, datos)
print("Error con validación cruzada con laplace: " + str(mio_tic_cruzada_laplace))

Error con validación simple sin la place: 0.2945993031358885
Error con validación simple sin la place: 0.29463414634146357
Error con validación cruzada: 0.3036805555555555
Error con validación cruzada: 0.304375


## German

In [5]:
datos = Datos('./ConjuntoDatos/german.data')
# Validacion simple con 20 iteraciones
vs = ValidacionSimple(0.30, 20)
# Validación simple con 5 folds
vc = ValidacionCruzada(5)
# Creación del clasificador
nb = ClasificadorNaiveBayes()
# Cálculo de errores
mio_german_simple = nb.validacion(vs, {"nombre": "naiveBayes", "laplace": False}, datos)
print("Error con validación simple sin laplace: " + str(mio_tic_simple))
mio_german_simple_laplace = nb.validacion(vs, {"nombre": "naiveBayes", "laplace": True}, datos)
print("Error con validación simple con laplace: " + str(mio_tic_simple_laplace))
mio_german_cruzada = nb.validacion(vc, {"nombre": "naiveBayes", "laplace": False}, datos)
print("Error con validación cruzada sin laplace: " + str(mio_tic_cruzada))
mio_german_cruzada_laplace = nb.validacion(vc, {"nombre": "naiveBayes", "laplace": True}, datos)
print("Error con validación cruzada con laplace: " + str(mio_german_cruzada_laplace))

Error con validación simple sin la place: 0.29515679442508713
Error con validación simple sin la place: 0.2941114982578398
Error con validación cruzada: 0.30576388888888884
Error con validación cruzada: 0.30576388888888884


# Scikit Learn
## Tic-tac-toe

In [9]:
data = pd.read_csv('./ConjuntoDatos/tic-tac-toe.data')

# One hot encoding all columns but the class column
X_data = pd.get_dummies(data.iloc[:, :-1])
Y_data = data.iloc[:, -1]

#### Split dataset for simple validation
xTrain, xTest, yTrain, yTest = train_test_split(X_data, Y_data, test_size=0.3)
clf_s = GaussianNB()
# Train classifier
score = 0
part = 1000
for _ in range(part):
    xTrain, xTest, yTrain, yTest = train_test_split(X_data, Y_data, test_size=0.3)
    # Train classifier
    clf_s.fit(xTrain, yTrain)
    # Validate classifier
    score += clf_s.score(xTest, yTest)
sk_tic_simple = 1 - score / part
print("Error con validación simple:" + str(sk_tic_simple))

#### Same calculation for cross validation wit 60 folds
clf_c = GaussianNB()
scores = cross_val_score(clf_c, X_data, Y_data, cv=50)
sk_tic_cruzada = (1 - scores).mean()
print("Error con validación cruzada:" + str(sk_tic_cruzada))


Error con validación simple:0.32590624999999973
Error con validación cruzada:0.3542105263157895


## German

In [8]:
data = pd.read_csv('./ConjuntoDatos/german.data')

# One hot encoding all columns but the class column
X_data = pd.get_dummies(data.iloc[:, :-1])
Y_data = data.iloc[:, -1]

clf_s = GaussianNB()

score = 0
part = 1000
for _ in range(part):
    xTrain, xTest, yTrain, yTest = train_test_split(X_data, Y_data, test_size=0.3)
    # Train classifier
    clf_s.fit(xTrain, yTrain)
    # Validate classifier
    score += clf_s.score(xTest, yTest)
sk_german_simple = 1 - score / part
print("Error con validación simple:" + str(sk_german_simple))

#### Same calculation for cross validation wit 60 folds
clf_c = GaussianNB()
scores = cross_val_score(clf_c, X_data, Y_data, cv=60)
sk_german_cruzada = (1 - scores).mean()
print("Error con validación cruzada:" + str(sk_german_cruzada))

Error con validación simple:0.28739999999999655
Error con validación cruzada:0.2810049019607843


# Análisis de resultados
Se puede observar que la implementación propia ha obtenido mejores resultados que la obtenida con sklearn.
Creemos que esto puede ser debido a que en sklearn hemos usado one hot encoding para pasar convertir los atributos nominales en contínuos. Quizás esta no sea la mejor opción debido a que Naive Bayes asume independencia entre coumnas. Sin embargo, en el caso del Tic Tac Toe, se crearían 3 nuevos atributos por cada atributo primitivo (1 para x, otro para o y otro para b), y estos están claramente relacionados, pues solo 1 de ellos puede ser 1 y el resto deben ser 0.


Teniendo en cuenta eso anterior, nuestra hipótesis es que ese error que inducimos en el clasificador más el propio error de el algoritmo Gaussiano causan que el error sea mayor que en el algorítmo implementado por nosotros.

Como alternativa a esta implementación en sklearn hay dos que creo que serían más eficaces:

1. Utilizar label encoding en vez de one hot encoding. De esta manera eliminaríamos el problema de la relación entre subatributos provenientes de un atributo primitivo. Esta posibilidad creemos que podría funcionar mejor que nuestra implementación. Sin embargo no tenemos seguridad total.

2. Utilizar ambos MultinomialNB y GaussianNB: De esta forma se puede entrenar los atributos categóricos con el clasificador multinomial y los continuos con el gausiano. Una vez se tengan calculados se crea una tabla de probabilidades para cada uno de ellos utilizando 'predict_proba' y posterior mente uniendo las 2 matrices de probabilidades en un único dataframe. Lo único que faltaría para concluir sería entrenar un clasificador de Bayes Gaussiano con este último dataframe. De esta forma creemos que se obtendría menor error en la clasificación de variables nominales al principio, pero tampoco sabemos con certeza como afectaría el entrenamiento del clasificador final. 
